In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
%matplotlib inline

movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')
movies.head()
credits.head()
movies = movies.merge(credits, on='title')
movies.info()
movies.isnull().sum()
movies = movies[['movie_id','title','genres','original_language','release_date','overview','runtime','keywords','cast','crew','vote_average']]
movies.info()
movies.isnull().sum()
# Deleting the row since it is only 3 null value

movies.dropna(inplace=True)
movies.isnull().sum()
movies.head(5)
n = 500

print(movies.iloc[n].title)
print(movies.iloc[n].movie_id)
print(movies.iloc[n].genres)
print(movies.iloc[n].overview)
print(movies.iloc[n].crew)
print(movies.iloc[n].cast)
print(movies.iloc[n].original_language)
print(movies.iloc[n].release_date)
print(movies.iloc[n].runtime)
print(movies.iloc[n].vote_average)
print(movies.iloc[n].keywords)
n = 500

print("MOVIE NAME:", movies.iloc[n].title, end='\n')
print("MOVIE ID:", movies.iloc[n].movie_id, end='\n')
print("GENRES:", movies.iloc[n].genres, end='\n')
print("OVERVIEW:", movies.iloc[n].overview, end='\n')
print(" ")
print("DIRECTED BY:",movies.iloc[n].crew, end='\n')
print(" ")
print("CAST:",movies.iloc[n].cast, end='\n')
print("ORIGINAL LANGUAGE:", movies.iloc[n].original_language, end='\n')
print("RELEASE DATE:", movies.iloc[n].release_date, end='\n')
print("RUNTIME:", movies.iloc[n].runtime, end='\n')
print("VOTE AVERAGE:", movies.iloc[n].vote_average, end='\n')
print("KEYWORDS:", movies.iloc[n].keywords, end='\n')

# genres column
movies.iloc[0].genres

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [2]:
#CONVERT GENRE INTO ['Action','Adventure','Fantasy','Science Fiction'] format
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L
movies['genres'] = movies['genres'].apply(convert)
# genres column
movies.iloc[0].genres
# keywords column
movies.iloc[0].keywords
movies['keywords'] = movies['keywords'].apply(convert)
movies.iloc[0].keywords
# cast column
movies.iloc[0].cast
# Taking top 3 name from cast

def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L 
movies['cast'] = movies['cast'].apply(convert3)
movies.iloc[0].cast
movies.iloc[0].crew
# Taking only director from crew only 

def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L
movies['crew'] = movies['crew'].apply(fetch_director)   
movies.iloc[0].crew
# Checking Overview column
movies['overview'][0]
# Splitting overview
movies['overview'] = movies['overview'].apply(lambda x:x.split())
# Checking Overview column
movies['overview'][0]
movies.info()
movies.iloc[0].runtime
movies['runtime'] = movies['runtime'].apply(lambda x: str(x))
movies.iloc[0].runtime
movies.iloc[0].vote_average
movies['vote_average'] = movies['vote_average'].apply(lambda x: str(x))
movies.iloc[0].vote_average
movies.info()
movies.head()
movies.iloc[0].release_date
# Changing 
# Sam Worthington --> SamWorthington
# Science Fiction --> ScienceFiction
# culture clash --> cultureclash
# because it will treat Sam Worthington and Sam Waltz as a same

movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['release_date']=movies['release_date'].apply(lambda x:[i.replace(" ","") for i in x])
movies['runtime']=movies['runtime'].apply(lambda x:[i.replace(" ","") for i in x])
movies['vote_average']=movies['vote_average'].apply(lambda x:[i.replace(" ","") for i in x])
movies.head()
movies['tags'] = movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']
new_df = movies[['movie_id','title','tags','genres','original_language','release_date','overview',
                 'runtime','cast','crew','vote_average']]
new_df.head()
# Removing commas from tags columns

new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))
new_df['tags'][0]
# Lower case

new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
new_df.head()
new_df['overview'][0]

<class 'pandas.core.frame.DataFrame'>
Index: 4805 entries, 0 to 4808
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie_id           4805 non-null   int64  
 1   title              4805 non-null   object 
 2   genres             4805 non-null   object 
 3   original_language  4805 non-null   object 
 4   release_date       4805 non-null   object 
 5   overview           4805 non-null   object 
 6   runtime            4805 non-null   float64
 7   keywords           4805 non-null   object 
 8   cast               4805 non-null   object 
 9   crew               4805 non-null   object 
 10  vote_average       4805 non-null   float64
dtypes: float64(2), int64(1), object(8)
memory usage: 579.5+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 4805 entries, 0 to 4808
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   movie_id 

C:\Users\ASUS\AppData\Local\Temp\ipykernel_13000\3293413110.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))
C:\Users\ASUS\AppData\Local\Temp\ipykernel_13000\3293413110.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


['In',
 'the',
 '22nd',
 'century,',
 'a',
 'paraplegic',
 'Marine',
 'is',
 'dispatched',
 'to',
 'the',
 'moon',
 'Pandora',
 'on',
 'a',
 'unique',
 'mission,',
 'but',
 'becomes',
 'torn',
 'between',
 'following',
 'orders',
 'and',
 'protecting',
 'an',
 'alien',
 'civilization.']

In [3]:

# Assuming new_df is already defined and loaded with data

# Removing commas from tags column
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x.split(',')))

# Displaying movie details
n = 500
print("MOVIE NAME:", new_df.iloc[n].title)
print("MOVIE ID:", new_df.iloc[n].movie_id)
print("GENRES:", new_df.iloc[n].genres)
print("OVERVIEW:", new_df.iloc[n].overview)
print("DIRECTED BY:", new_df.iloc[n].crew)
print("CAST:", new_df.iloc[n].cast)
print("ORIGINAL LANGUAGE:", new_df.iloc[n].original_language)
print("RELEASE DATE:", new_df.iloc[n].release_date)
print("RUNTIME:", new_df.iloc[n].runtime)
print("VOTE AVERAGE:", new_df.iloc[n].vote_average)

# Transforming text into vectors
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()
print(vectors[0])
feature_names = cv.get_feature_names_out()
print(feature_names)

# Using stemming for same value
ps = PorterStemmer()
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

new_df['tags'] = new_df['tags'].apply(stem)

# Transforming text into vectors again after stemming
vectors = cv.fit_transform(new_df['tags']).toarray()
feature_names = cv.get_feature_names_out()
print(len(feature_names))

# Using cosine similarity
similarity = cosine_similarity(vectors)
print(sorted(list(enumerate(similarity[1])), reverse=True, key=lambda x: x[1])[1:6])


MOVIE NAME: 2 Fast 2 Furious
MOVIE ID: 584
GENRES: ['Action', 'Crime', 'Thriller']
OVERVIEW: ["It's", 'a', 'major', 'double-cross', 'when', 'former', 'police', 'officer', 'Brian', "O'Conner", 'teams', 'up', 'with', 'his', 'ex-con', 'buddy', 'Roman', 'Pearce', 'to', 'transport', 'a', 'shipment', 'of', '"dirty"', 'money', 'for', 'shady', 'Miami-based', 'import-export', 'dealer', 'Carter', 'Verone.', 'But', 'the', 'guys', 'are', 'actually', 'working', 'with', 'undercover', 'agent', 'Monica', 'Fuentes', 'to', 'bring', 'Verone', 'down.']
DIRECTED BY: ['JohnSingleton']
CAST: ['PaulWalker', 'TyreseGibson', 'EvaMendes']
ORIGINAL LANGUAGE: en
RELEASE DATE: ['2', '0', '0', '3', '-', '0', '6', '-', '0', '5']
RUNTIME: ['1', '0', '7', '.', '0']
VOTE AVERAGE: ['6', '.', '2']


C:\Users\ASUS\AppData\Local\Temp\ipykernel_13000\858477512.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x.split(',')))


[0 0 0 ... 0 0 0]
['000' '007' '10' ... 'zone' 'zoo' 'zooeydeschanel']


C:\Users\ASUS\AppData\Local\Temp\ipykernel_13000\858477512.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


5000
[(12, 0.4140043409440133), (199, 0.27500954910846337), (17, 0.2552613778869102), (216, 0.20180183819889375), (3570, 0.19446111706564928)]


In [4]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    for i in movies_list:
        movie_info = new_df.iloc[i[0]]
        print("MOVIE NAME:", movie_info.title)
        print("GENRES:", ', '.join(movie_info.genres))
        print("OVERVIEW:", movie_info.overview)
        
        # Join crew names with space between first and last names
        directed_by = ' '.join(movie_info.crew)
        print("DIRECTED BY:", directed_by)
        
        # Join cast names with space between first and last names
        cast_names = ', '.join([name.replace(" ", " ") for name in movie_info.cast])
        print("CAST:", cast_names)
        
        print("ORIGINAL LANGUAGE:", movie_info.original_language)
        print("RELEASE DATE:", ''.join(movie_info.release_date))
        
        # Convert runtime to hours and minutes
        runtime_minutes = float(''.join(movie_info.runtime))
        runtime_hours = int(runtime_minutes // 60)
        runtime_minutes %= 60
        print("RUNTIME:", f"{runtime_hours} hours {int(runtime_minutes)} minutes")
        
        print("VOTE AVERAGE:", ''.join(movie_info.vote_average))
        print("\n")

recommend('Batman Begins')

pickle.dump(new_df.to_dict(),open('movie_dict_4.pkl','wb'))
pickle.dump(similarity,open('similarity_4.pkl','wb'))

MOVIE NAME: The Dark Knight
GENRES: Drama, Action, Crime, Thriller
OVERVIEW: ['Batman', 'raises', 'the', 'stakes', 'in', 'his', 'war', 'on', 'crime.', 'With', 'the', 'help', 'of', 'Lt.', 'Jim', 'Gordon', 'and', 'District', 'Attorney', 'Harvey', 'Dent,', 'Batman', 'sets', 'out', 'to', 'dismantle', 'the', 'remaining', 'criminal', 'organizations', 'that', 'plague', 'the', 'streets.', 'The', 'partnership', 'proves', 'to', 'be', 'effective,', 'but', 'they', 'soon', 'find', 'themselves', 'prey', 'to', 'a', 'reign', 'of', 'chaos', 'unleashed', 'by', 'a', 'rising', 'criminal', 'mastermind', 'known', 'to', 'the', 'terrified', 'citizens', 'of', 'Gotham', 'as', 'the', 'Joker.']
DIRECTED BY: ChristopherNolan
CAST: ChristianBale, HeathLedger, AaronEckhart
ORIGINAL LANGUAGE: en
RELEASE DATE: 2008-07-16
RUNTIME: 2 hours 32 minutes
VOTE AVERAGE: 8.2


MOVIE NAME: Batman
GENRES: Fantasy, Action
OVERVIEW: ['The', 'Dark', 'Knight', 'of', 'Gotham', 'City', 'begins', 'his', 'war', 'on', 'crime', 'with', 'h